# Module 06: Forms & Validation

**Estimated Time:** 2.5 hours  
**Difficulty:** ⭐⭐

---

## Prerequisites

Before starting this module, ensure you've completed:

- ✅ **Module 00-04: Django fundamentals**
- ✅ **Module 05: Templates**
- ✅ **Understanding of HTML forms**

---

## Learning Objectives

By the end of this module, you will:

- ✅ Create Django forms
- ✅ Use ModelForms for database models
- ✅ Implement form validation
- ✅ Handle form submissions in views
- ✅ Display form errors
- ✅ Understand CSRF protection
- ✅ Create post creation and editing forms

---

## 1. Introduction to Django Forms

Django forms provide:
- Automatic HTML generation
- Data validation
- Security (CSRF protection)
- Error handling
- Cleaned data

### Two Types of Forms
1. **Form**: General-purpose forms
2. **ModelForm**: Forms tied to models (auto-generated)

In [ ]:
from pathlib import Path

# Setup paths
notebook_dir = Path.cwd()
project_path = notebook_dir.parent / "projects" / "myblog"
blog_app = project_path / "blog"
forms_file = blog_app / "forms.py"

print(f"Project: {project_path}")
print(f"Forms file: {forms_file}")

## 2. Creating ModelForms

ModelForms automatically generate form fields from model fields.

In [ ]:
# Create forms.py
forms_code = '''from django import forms
from django.utils.text import slugify
from .models import Post, Category


class PostForm(forms.ModelForm):
    """Form for creating and editing blog posts"""
    
    class Meta:
        model = Post
        fields = ['title', 'slug', 'content', 'categories', 'status', 'publish_date']
        widgets = {
            'title': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Enter post title'
            }),
            'slug': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'url-friendly-slug'
            }),
            'content': forms.Textarea(attrs={
                'class': 'form-control',
                'rows': 10,
                'placeholder': 'Write your post content here...'
            }),
            'categories': forms.CheckboxSelectMultiple(),
            'status': forms.Select(attrs={'class': 'form-control'}),
            'publish_date': forms.DateTimeInput(attrs={
                'class': 'form-control',
                'type': 'datetime-local'
            }),
        }
        help_texts = {
            'slug': 'Leave blank to auto-generate from title',
            'categories': 'Select one or more categories',
        }
    
    def clean_slug(self):
        """Auto-generate slug if not provided"""
        slug = self.cleaned_data.get('slug')
        if not slug:
            title = self.cleaned_data.get('title')
            if title:
                slug = slugify(title)
        
        # Check for duplicate slugs
        if Post.objects.filter(slug=slug).exclude(pk=self.instance.pk).exists():
            raise forms.ValidationError('A post with this slug already exists.')
        
        return slug
    
    def clean_title(self):
        """Validate title length"""
        title = self.cleaned_data.get('title')
        if len(title) < 5:
            raise forms.ValidationError('Title must be at least 5 characters long.')
        return title


class CategoryForm(forms.ModelForm):
    """Form for creating categories"""
    
    class Meta:
        model = Category
        fields = ['name', 'slug', 'description']
        widgets = {
            'name': forms.TextInput(attrs={'class': 'form-control'}),
            'slug': forms.TextInput(attrs={'class': 'form-control'}),
            'description': forms.Textarea(attrs={
                'class': 'form-control',
                'rows': 3
            }),
        }
    
    def clean_slug(self):
        slug = self.cleaned_data.get('slug')
        if not slug:
            slug = slugify(self.cleaned_data.get('name', ''))
        return slug


class ContactForm(forms.Form):
    """Simple contact form (not tied to a model)"""
    
    name = forms.CharField(
        max_length=100,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Your name'
        })
    )
    email = forms.EmailField(
        widget=forms.EmailInput(attrs={
            'class': 'form-control',
            'placeholder': 'your@email.com'
        })
    )
    subject = forms.CharField(
        max_length=200,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Subject'
        })
    )
    message = forms.CharField(
        widget=forms.Textarea(attrs={
            'class': 'form-control',
            'rows': 5,
            'placeholder': 'Your message...'
        })
    )
    
    def clean_email(self):
        email = self.cleaned_data.get('email')
        if not email.endswith(('.com', '.org', '.net')):
            raise forms.ValidationError('Please use a valid email domain.')
        return email
'''

with open(forms_file, "w") as f:
    f.write(forms_code)

print("✓ Forms created!")
print("\nForms:")
print("  - PostForm: Create/edit posts")
print("  - CategoryForm: Create categories")
print("  - ContactForm: Contact page")

## 3. Form Handling in Views

Let's create views that handle form submission.

In [ ]:
# Add form views to views.py
views_file = blog_app / "views.py"

with open(views_file, "r") as f:
    current_views = f.read()

# Add form-handling views
form_views = '''

# Form handling views
from django.shortcuts import redirect
from django.contrib import messages
from django.views.generic.edit import CreateView, UpdateView, DeleteView
from django.urls import reverse_lazy
from .forms import PostForm, CategoryForm


class PostCreateView(CreateView):
    """Create new post"""
    model = Post
    form_class = PostForm
    template_name = 'blog/post_form.html'
    success_url = reverse_lazy('blog:home')
    
    def form_valid(self, form):
        form.instance.author = self.request.user
        messages.success(self.request, 'Post created successfully!')
        return super().form_valid(form)


class PostUpdateView(UpdateView):
    """Edit existing post"""
    model = Post
    form_class = PostForm
    template_name = 'blog/post_form.html'
    
    def get_success_url(self):
        return reverse_lazy('blog:post_detail', kwargs={'slug': self.object.slug})
    
    def form_valid(self, form):
        messages.success(self.request, 'Post updated successfully!')
        return super().form_valid(form)


class PostDeleteView(DeleteView):
    """Delete post"""
    model = Post
    template_name = 'blog/post_confirm_delete.html'
    success_url = reverse_lazy('blog:home')
    
    def delete(self, request, *args, **kwargs):
        messages.success(self.request, 'Post deleted successfully!')
        return super().delete(request, *args, **kwargs)
'''

with open(views_file, "w") as f:
    f.write(current_views + form_views)

print("✓ Form views added!")

## 4. Form Templates

In [ ]:
# Create form template
templates_dir = blog_app / "templates" / "blog"
templates_dir.mkdir(parents=True, exist_ok=True)

post_form_template = """{% extends 'blog/base.html' %}

{% block title %}{% if object %}Edit{% else %}Create{% endif %} Post{% endblock %}

{% block content %}
<h2>{% if object %}Edit{% else %}Create New{% endif %} Post</h2>

<form method="post">
    {% csrf_token %}
    
    {% if form.non_field_errors %}
    <div class="errors">
        {{ form.non_field_errors }}
    </div>
    {% endif %}
    
    {% for field in form %}
    <div class="form-group">
        <label for="{{ field.id_for_label }}">{{ field.label }}</label>
        {{ field }}
        
        {% if field.help_text %}
        <small class="form-text">{{ field.help_text }}</small>
        {% endif %}
        
        {% if field.errors %}
        <div class="field-errors">
            {{ field.errors }}
        </div>
        {% endif %}
    </div>
    {% endfor %}
    
    <button type="submit" class="btn">{% if object %}Update{% else %}Create{% endif %} Post</button>
    <a href="{% url 'blog:home' %}" class="btn-cancel">Cancel</a>
</form>

<style>
.form-group {
    margin-bottom: 1rem;
}
.form-group label {
    display: block;
    font-weight: bold;
    margin-bottom: 0.5rem;
}
.form-control {
    width: 100%;
    padding: 0.5rem;
    border: 1px solid #ddd;
    border-radius: 4px;
}
.btn {
    background: #007bff;
    color: white;
    padding: 0.5rem 1rem;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}
.field-errors {
    color: red;
    font-size: 0.9em;
    margin-top: 0.25rem;
}
</style>
{% endblock %}
"""

with open(templates_dir / "post_form.html", "w") as f:
    f.write(post_form_template)

# Delete confirmation template
delete_template = """{% extends 'blog/base.html' %}

{% block title %}Delete Post{% endblock %}

{% block content %}
<h2>Delete Post</h2>

<p>Are you sure you want to delete "{{ object.title }}"?</p>

<form method="post">
    {% csrf_token %}
    <button type="submit" class="btn-danger">Yes, delete</button>
    <a href="{% url 'blog:post_detail' object.slug %}" class="btn">Cancel</a>
</form>
{% endblock %}
"""

with open(templates_dir / "post_confirm_delete.html", "w") as f:
    f.write(delete_template)

print("✓ Form templates created!")

## 5. Update URLs

In [ ]:
# Update URLs to include form views
urls_file = blog_app / "urls.py"

urls_code = """from django.urls import path
from . import views

app_name = 'blog'

urlpatterns = [
    path('', views.HomeView.as_view(), name='home'),
    path('post/<slug:slug>/', views.PostDetailView.as_view(), name='post_detail'),
    path('category/<slug:slug>/', views.CategoryPostsView.as_view(), name='category_posts'),
    path('about/', views.AboutView.as_view(), name='about'),
    
    # CRUD operations
    path('post/create/', views.PostCreateView.as_view(), name='post_create'),
    path('post/<slug:slug>/edit/', views.PostUpdateView.as_view(), name='post_edit'),
    path('post/<slug:slug>/delete/', views.PostDeleteView.as_view(), name='post_delete'),
]
"""

with open(urls_file, "w") as f:
    f.write(urls_code)

print("✓ URLs updated with CRUD operations")

## 6. Form Validation

### Built-in Validators
```python
from django.core.validators import MinLengthValidator, MaxLengthValidator, EmailValidator

title = forms.CharField(
    validators=[MinLengthValidator(5)]
)
```

### Custom Validation
```python
def clean_fieldname(self):
    data = self.cleaned_data['fieldname']
    # Validation logic
    if some_condition:
        raise forms.ValidationError('Error message')
    return data
```

### Multi-field Validation
```python
def clean(self):
    cleaned_data = super().clean()
    field1 = cleaned_data.get('field1')
    field2 = cleaned_data.get('field2')
    
    if field1 and field2:
        # Cross-field validation
        pass
    
    return cleaned_data
```

## 7. CSRF Protection

Django protects against Cross-Site Request Forgery attacks.

### Always include in POST forms:
```django
<form method="post">
    {% csrf_token %}
    <!-- form fields -->
</form>
```

### What it does:
- Generates unique token per session
- Validates token on form submission
- Prevents malicious form submissions

### Exempt views (rarely needed):
```python
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def my_view(request):
    # Only for APIs or special cases
    pass
```

## 8. Form Rendering Methods

### As Paragraph (default)
```django
{{ form.as_p }}
```

### As Table
```django
<table>
    {{ form.as_table }}
</table>
```

### As List
```django
<ul>
    {{ form.as_ul }}
</ul>
```

### Manual Rendering (best control)
```django
{% for field in form %}
    <div class="field">
        {{ field.label_tag }}
        {{ field }}
        {{ field.errors }}
    </div>
{% endfor %}
```

## 9. Best Practices

### Form Design
1. Use ModelForms when possible
2. Add help_text for clarity
3. Use widgets for better UX
4. Validate on both client and server

### Security
1. Always use {% csrf_token %}
2. Validate all user input
3. Use cleaned_data, not raw POST data
4. Sanitize HTML input

### User Experience
1. Show clear error messages
2. Preserve form data on errors
3. Use placeholders and labels
4. Provide success feedback

## 8. Hands-On Practice

Master Django forms through these practical exercises!

### Exercise 1: Custom Form Validation ⭐

**Task**: Add custom validation to prevent certain words in post titles.

In your `PostForm`:
```python
class PostForm(forms.ModelForm):
    # ... fields ...
    
    def clean_title(self):
        title = self.cleaned_data.get('title')
        
        # Blacklist certain words
        forbidden_words = ['spam', 'click here', 'free money']
        
        for word in forbidden_words:
            if word.lower() in title.lower():
                raise forms.ValidationError(
                    f'Title cannot contain "{word}"'
                )
        
        # Must be at least 10 characters
        if len(title) < 10:
            raise forms.ValidationError(
                'Title must be at least 10 characters long'
            )
        
        return title
```

**Test it**: Try submitting a form with a title containing "spam".

---

### Exercise 2: Create a Contact Form ⭐⭐

**Task**: Build a contact form that sends an email.

1. **Create the form** (`blog/forms.py`):
```python
class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    subject = forms.CharField(max_length=200)
    message = forms.CharField(widget=forms.Textarea)
    
    def clean_message(self):
        message = self.cleaned_data.get('message')
        if len(message) < 10:
            raise forms.ValidationError(
                'Message must be at least 10 characters'
            )
        return message
```

2. **Create the view** (`blog/views.py`):
```python
from django.core.mail import send_mail

def contact(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            # Process form data
            name = form.cleaned_data['name']
            email = form.cleaned_data['email']
            subject = form.cleaned_data['subject']
            message = form.cleaned_data['message']
            
            # Send email (configure email settings first)
            send_mail(
                subject,
                f'From: {name} ({email})\n\n{message}',
                email,
                ['your-email@example.com'],
            )
            
            messages.success(request, 'Message sent successfully!')
            return redirect('home')
    else:
        form = ContactForm()
    
    return render(request, 'blog/contact.html', {'form': form})
```

3. **Create the template** (`templates/blog/contact.html`):
```django
{% extends 'base.html' %}

{% block content %}
<h2>Contact Us</h2>

<form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Send Message</button>
</form>
{% endblock %}
```

---

### Exercise 3: Enhanced Form Display with Error Handling ⭐⭐

**Task**: Create a better form template with proper error display.

```django
<form method="post" class="blog-form">
    {% csrf_token %}
    
    {% if form.non_field_errors %}
        <div class="alert alert-error">
            {{ form.non_field_errors }}
        </div>
    {% endif %}
    
    {% for field in form %}
        <div class="form-group {% if field.errors %}has-error{% endif %}">
            <label for="{{ field.id_for_label }}">
                {{ field.label }}
                {% if field.field.required %}<span class="required">*</span>{% endif %}
            </label>
            
            {{ field }}
            
            {% if field.help_text %}
                <small class="help-text">{{ field.help_text }}</small>
            {% endif %}
            
            {% if field.errors %}
                <div class="error-message">
                    {% for error in field.errors %}
                        <p>{{ error }}</p>
                    {% endfor %}
                </div>
            {% endif %}
        </div>
    {% endfor %}
    
    <button type="submit">Submit</button>
</form>
```

Add CSS to style errors appropriately.

---

### Exercise 4: Multi-Step Form with Sessions ⭐⭐⭐

**Task**: Create a multi-step post creation wizard.

**Step 1 - Basic Info**:
```python
class PostBasicInfoForm(forms.Form):
    title = forms.CharField(max_length=200)
    slug = forms.SlugField(max_length=200)
```

**Step 2 - Content**:
```python
class PostContentForm(forms.Form):
    content = forms.CharField(widget=forms.Textarea)
    excerpt = forms.CharField(widget=forms.Textarea, required=False)
```

**Step 3 - Settings**:
```python
class PostSettingsForm(forms.Form):
    status = forms.ChoiceField(choices=[('draft', 'Draft'), ('published', 'Published')])
    categories = forms.ModelMultipleChoiceField(queryset=Category.objects.all())
```

**View logic**:
```python
def create_post_step1(request):
    if request.method == 'POST':
        form = PostBasicInfoForm(request.POST)
        if form.is_valid():
            request.session['post_step1'] = form.cleaned_data
            return redirect('create_post_step2')
    else:
        form = PostBasicInfoForm()
    return render(request, 'blog/create_post_step1.html', {'form': form})

# Similar views for step2 and step3
# In final step, combine all session data and create post
```

---

### Exercise 5: Form with File Upload Validation ⭐⭐

**Task**: Add image upload with validation.

```python
class PostWithImageForm(forms.ModelForm):
    image = forms.ImageField(required=False)
    
    class Meta:
        model = Post
        fields = ['title', 'content', 'image']
    
    def clean_image(self):
        image = self.cleaned_data.get('image')
        
        if image:
            # Check file size (max 2MB)
            if image.size > 2 * 1024 * 1024:
                raise forms.ValidationError(
                    'Image file too large (max 2MB)'
                )
            
            # Check file type
            if not image.content_type in ['image/jpeg', 'image/png', 'image/gif']:
                raise forms.ValidationError(
                    'Only JPEG, PNG, and GIF images are allowed'
                )
        
        return image
```

**Template** (note enctype):
```django
<form method="post" enctype="multipart/form-data">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Upload</button>
</form>
```

---

### Bonus: Formsets for Multiple Objects ⭐⭐⭐

**Task**: Allow editing multiple objects at once.

```python
from django.forms import modelformset_factory

# Create a formset for Categories
CategoryFormSet = modelformset_factory(
    Category,
    fields=('name', 'slug', 'description'),
    extra=2,  # Number of empty forms to display
    can_delete=True
)

def manage_categories(request):
    if request.method == 'POST':
        formset = CategoryFormSet(request.POST)
        if formset.is_valid():
            formset.save()
            messages.success(request, 'Categories updated!')
            return redirect('manage_categories')
    else:
        formset = CategoryFormSet(queryset=Category.objects.all())
    
    return render(request, 'blog/manage_categories.html', {
        'formset': formset
    })
```

**Template**:
```django
<form method="post">
    {% csrf_token %}
    {{ formset.management_form }}
    
    {% for form in formset %}
        <div class="form-row">
            {{ form.as_p }}
        </div>
    {% endfor %}
    
    <button type="submit">Save All</button>
</form>
```

---

### 💡 Tips for Success

- ✅ Always validate on the server side (never trust client data)
- ✅ Use `cleaned_data` to access validated form data
- ✅ Provide clear, helpful error messages
- ✅ Test edge cases (empty data, special characters, etc.)
- ✅ Use CSRF protection on all forms

**Master these exercises and you'll handle any form requirement!**

---


## 10. Summary & Next Steps

### What We Accomplished

✅ Created ModelForms for Post and Category  
✅ Built form handling views (Create, Update, Delete)  
✅ Implemented form validation  
✅ Created form templates  
✅ Added CSRF protection  
✅ Updated URLs for CRUD operations  

### Forms Created
- PostForm: Create and edit blog posts
- CategoryForm: Manage categories
- ContactForm: Simple contact form example


### Quick Check

Before moving on, ensure you can answer:

1. What's the difference between Form and ModelForm?
2. What is CSRF protection and why is it important?
3. How do you display form errors in templates?
4. What method validates a specific field in a form?

### What's Next

In **Module 07**, we'll:
- Configure static files (CSS, JavaScript)
- Add custom styling to our blog
- Handle media file uploads
- Organize assets properly

---

**Great work! Your blog now has full CRUD functionality. Continue to Module 07!** 📝